<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/classification/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [2]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

#Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [17]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.9, svd_solver='full')

In [21]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [23]:
train_data_stand_pca.shape

(131, 9)

#LDA

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [0]:
lda = LinearDiscriminantAnalysis()

In [26]:
lda.fit(train_data_stand, train_labels_dec)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [0]:
train_data_stand_lda = lda.transform(train_data_stand)
test_data_stand_lda = lda.transform(test_data_stand)

In [28]:
train_data_stand_lda.shape

(131, 2)

#Train Test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(train_data_stand_pca, train_labels_dec,
                                                    stratify=train_labels_dec,
                                                    test_size=0.30,
                                                    random_state=3)

In [0]:
X_train_lda, X_val_lda, y_train_lda, y_val_lda = train_test_split(train_data_stand_lda, train_labels_dec,
                                                    stratify=train_labels_dec,
                                                    test_size=0.30,
                                                    random_state=3)


In [0]:
#y_train.count(2)

#Built classification algorithm: Nearest Neighbors

In [0]:
from sklearn.neighbors import KNeighborsClassifier


In [0]:
neigh = KNeighborsClassifier(n_neighbors=10)

In [35]:
neigh.fit(X_train_pca, y_train_pca)
neigh.score(X_val_pca, y_val_pca)

0.45

In [36]:
neigh.score(test_data_stand_pca, test_labels_dec)

0.20588235294117646

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = neigh.predict(X_val_pca)

cm = confusion_matrix(y_val_pca, y_pred)
print(cm)
print('Accuracy' + str(accuracy_score(y_val_pca, y_pred)))

[[ 1  2  2]
 [ 0 12  7]
 [ 0 11  5]]
Accuracy0.45


In [38]:
neigh.fit(X_train_lda, y_train_lda)
neigh.score(X_val_lda, y_val_lda)

0.975

In [39]:
neigh.score(test_data_stand_lda, test_labels_dec)

0.14705882352941177

In [40]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = neigh.predict(X_val_lda)

cm = confusion_matrix(y_val_lda, y_pred)
print(cm)
print('Accuracy' + str(accuracy_score(y_val_lda, y_pred)))

[[ 5  0  0]
 [ 0 19  0]
 [ 0  1 15]]
Accuracy0.975


#Built classification algorithm: SVM

In [0]:
from sklearn.svm import SVC

In [0]:
clf = SVC(gamma='auto')

In [43]:
clf.fit(X_train_pca, y_train_pca)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [44]:
clf.score(X_val_pca, y_val_pca)

0.45

In [45]:
clf.score(test_data_stand_pca, test_labels_dec)

0.11764705882352941

In [46]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_val_pca)

cm = confusion_matrix(y_val_pca, y_pred)
print(cm)
print('Accuracy' + str(accuracy_score(y_val_pca, y_pred)))

[[ 0  3  2]
 [ 0 12  7]
 [ 0 10  6]]
Accuracy0.45


In [47]:
clf.fit(X_train_lda, y_train_lda)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [48]:
clf.score(X_val_lda, y_val_lda)

0.95

In [49]:
clf.score(test_data_stand_lda, test_labels_dec)

0.29411764705882354

In [50]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_val_lda)

cm = confusion_matrix(y_val_lda, y_pred)
print(cm)
print('Accuracy' + str(accuracy_score(y_val_lda, y_pred)))

[[ 4  0  1]
 [ 0 19  0]
 [ 0  1 15]]
Accuracy0.95
